<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#1.1-Обработка-строк-и-лемматизация" data-toc-modified-id="1.1-Обработка-строк-и-лемматизация-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>1.1 Обработка строк и лемматизация</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

В данном проекте нам предстоит помочь интернет-магазину 'Викишоп' классифицировать комментарии на позитивные и негативные.

Мы будем работать с датафреймом, состоящим из двух колонок - сам комментарий и его оценка - позитивный/негативный.

Для начала изучим данные и проведем их подготовку: проверим на пропуски и дубликаты, и при необходимости исправим. После этого удалим все лишние знаки, оставив только буквы. И в конце лемматизируем комментарии.

После этого мы разобьем наши данные на выборки. Закодируем признак (сами комментарии) и при помощи различных моделей проведем моделирование. В качестве метрики качества будем использовать метрику f1. Ее значение должно быть не менее 0.75.



In [1]:
!pip install nltk

In [2]:
import numpy as np
import pandas as pd
import re

from tqdm import tqdm
tqdm.pandas()

import spacy
from spacy.lang.en import stop_words


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.pipeline import Pipeline

In [3]:
RANDOM_STATE = 42

## Подготовка

Загрузим наш датасет. И посмотрим информацию о нем.

In [4]:
try:
    toxic_comments = pd.read_csv('/datasets/toxic_comments.csv', index_col='Unnamed: 0')
except:
    toxic_comments = pd.read_csv('C/datasets/toxic_comments.csv', index_col='Unnamed: 0')

In [5]:
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [6]:
toxic_comments.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Датафрейм содержит две колонки, целевой признак представлен типом int64, названия колонок соответствуют змеиному регистру. Пропуски отсутствтуют. Данные успешно загружены.

Теперь проверим наши данные на наличие дубликатов, и если такие есть, то удалим их.

In [7]:
toxic_comments.duplicated().sum()

0

Дубликаты отсутствуют.

Также посмотрим на пропорцию наших данных, сколько комментариев токсичных, а сколько нет. Это нам поможет в дальнейшем при создании модели и разбиении данных на выборки.

In [8]:
toxic_comments['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Получается, что только 10% комментатриев токсичные, а большинсвто нет. Тк одного класса намного больше, чем другого, то при разбиении данных на выборки будем использовать стратификацию по целевому признаку. 

### 1.1 Обработка строк и лемматизация 

Напишем функцию 'clear_text', которая оставит в тексте только буквы, а также переведет все символы в нижний регистр. Также уберем лишние пробелы.

In [9]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    return ' '.join(text)

Теперь напишем функцию для лемматизации. Будем использовать библиотеку SpaCy и модель "en_core_web_sm", тк текст у нас на английском языке. Разобьем строку на токены, далее лемматизируем каждое слово. И удалим из списка стоп-слова. Далее все соберем назад в одну строку и вернем.

In [10]:
# Загрузка модели
nlp = spacy.load("en_core_web_sm")
#загрузка стоп-слов
stop_words = stop_words.STOP_WORDS

In [11]:
def lemm(text):
    sentence = nlp(text)
    #лемматизация
    sentence = [word.lemma_ for word in sentence]
    #удаление стоп-слов
    sentence = [word for word in sentence if word not in stop_words]      
    return ' '.join(sentence)

Теперь применим наши функции к комментариям. Результат запишем в новой колонке 'lemm_text'

In [12]:
toxic_comments['lemm_text'] = toxic_comments['text'].progress_apply(clear_text)
toxic_comments['lemm_text'] = toxic_comments['lemm_text'].progress_apply(lemm)

100%|██████████| 159292/159292 [35:34<00:00, 74.63it/s] 


Комментарии подготовлены, слова в них лемматезированы и удалены стоп-слова. Далее можно переходить к энкодингу и обучению.

## Обучение

Для начала разобьем наши данные, на тренировочную и тестовую выборки, учитывая стратификацию

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    toxic_comments['lemm_text'], 
    toxic_comments['toxic'], 
    random_state = RANDOM_STATE,
    stratify = toxic_comments['toxic']
)

После того, как данные разделены, нужно векторизировать лемматизированный текст (признак). Для этого воспользуемся TfidfVectorizer().

In [14]:
#count_tf_idf = TfidfVectorizer(stop_words=list(stopwords))
#tfidf_train = count_tf_idf.fit_transform(X_train)
#tfidf_test = count_tf_idf.transform(X_test)

Теперь создадим пайплайн с различными моделями и параметрами, и при помощи RandomizedSearchCV() и метрики 'f1' найдем наилучшую модель. Так как перед нами задача классификации, то в качестве моделей будем использовать DecisionTreeClassifier(), KNeighborsClassifier() и LogisticRegression(). Условием прохождения будет являться метрика 'f1' >= 0.75

In [15]:
pipe_final = Pipeline([
    ('vect', TfidfVectorizer()),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

param_grid = [
     # словарь для модели KNeighborsClassifier() 
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(3, 20)
    },

    # словарь для модели DecisionTreeClassifier()
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 20),
        'models__max_features': range(2, 20) 
    },    
   
    # словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE, 
            solver='liblinear'
        )],
        'models__penalty': ['l1', 'l2'],
        'models__C': range(1, 20)
    }
]

randomized_search = RandomizedSearchCV(
    pipe_final, 
    param_grid, 
    cv=5,
    n_iter=20,
    scoring='f1',
    random_state=RANDOM_STATE,
    n_jobs=-1
)

randomized_search.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_)
print ('Метрика лучшей модели на тренировочной выборке:', randomized_search.best_score_)

Лучшая модель и её параметры:

 Pipeline(steps=[('vect', TfidfVectorizer()),
                ('models',
                 LogisticRegression(C=13, random_state=42,
                                    solver='liblinear'))])
Метрика лучшей модели на тренировочной выборке: 0.7764700623258702


Качество модели допустимое, поэтому теперь проверим нашу модель на тестовых данных.

In [16]:
y_test_pred = randomized_search.predict(X_test)
print(f'Метрика F1 на тестовой выборке: {f1_score(y_test, y_test_pred)}')

Метрика F1 на тестовой выборке: 0.7773812791655229


## Выводы

Мы успешно создали модель для классификации комментариев на позитивные и негативные. Метрика f1 составила 0.77, что является допуском. 
Лучше всего с данной задачей справилась модель LogisticRegression(C=13, random_state=42,  solver='liblinear')

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны